<a href="https://colab.research.google.com/github/JoeyMucci/SemanticSegmentationSatelliteImagery/blob/main/HyperparameterOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hpbandster # Hyperparameter Optimization
import logging
logging.basicConfig(level=logging.WARNING)
import argparse
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres
from hpbandster.optimizers import BOHB as BOHB
from hpbandster.examples.commons import MyWorker
import numpy as np
import time
import ConfigSpace as CS
from hpbandster.core.worker import Worker

import logging
import os
import cv2
!pip install patchify 
!pip install segmentation-models 
import tensorflow
from matplotlib import pyplot as plt
from patchify import patchify
from PIL import Image
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
from keras.metrics.metrics import PrecisionAtRecall
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import sys
path_m = '/content/drive/MyDrive/ColabNotebooks/'
sys.path.insert(0,path_m)
import simple_multi_unet_model
from simple_multi_unet_model import multi_unet_model, jacard_coef  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 114 kB/s 
     |████████████████████████████████| 89 kB 3.9 MB/s 
     |████████████████████████████████| 4.9 MB 45.4 MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80009 sha256=69f54e213e66bac12322f0178ab98337fc4ee3d5bfbf1369bfbba6cf2a55b1af
  Stored in directory: /root/.cache/pip/wheels/d9/88/fc/61ab6b9f386a386839668631c39a6dc3c2fb0ec7000d552faa
Successfully built hpbandster
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.0 MB/s 
Segmentation Models: using `keras` framework.


In [ ]:
scaler = MinMaxScaler()

root_directory = 'drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/'

patch_size = 256

# Read images into patches and crop
image_dataset = []  
for path, subdirs, files in os.walk(root_directory):
    subdirs.sort(); # sort the subdirectory so the images and masks match
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'images':   
        images = os.listdir(path)  # list of all image names in this subdirectory
        images.sort() # sort the images so they match with the masks
        for i, image_name in enumerate(images):  
            if image_name.endswith(".jpg"):              
                image = cv2.imread(path+"/"+image_name, 1)  

                # get the highest size divisble than 256 but less than the original size
                SIZE_X = (image.shape[1]//patch_size)*patch_size 
                SIZE_Y = (image.shape[0]//patch_size)*patch_size 

                image = Image.fromarray(image)
                image = image.crop((0 ,0, SIZE_X, SIZE_Y))  # crop from top left corner
                image = np.array(image)             
      
                # extract patches from each image
                print("Now patchifying image:", path+"/"+image_name)
                patches_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)
        
                for i in range(patches_img.shape[0]):
                    for j in range(patches_img.shape[1]):
                        single_patch_img = patches_img[i,j,:,:]
                        single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
                        single_patch_img = single_patch_img[0] # drop the extra unecessary dimension that patchify adds                         
                        image_dataset.append(single_patch_img)
                
# Read corresponding masks into patches and crop
mask_dataset = []  
for path, subdirs, files in os.walk(root_directory):
    subdirs.sort();
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'masks':   
        masks = os.listdir(path)  # list of all mask names in this subdirectory
        masks.sort(); # sort the masks so they match with the images
        for i, mask_name in enumerate(masks):  
            if mask_name.endswith(".png"):     
                mask = cv2.imread(path+"/"+mask_name, 1)  
                mask = cv2.cvtColor(mask,cv2.COLOR_BGR2RGB)

                # get the highest size divisble than 256 but less than the original size
                SIZE_X = (mask.shape[1]//patch_size)*patch_size 
                SIZE_Y = (mask.shape[0]//patch_size)*patch_size 

                mask = Image.fromarray(mask)
                mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
                mask = np.array(mask)             
      
                # extract patches from each image
                print("Now patchifying mask:", path+"/"+mask_name)
                patches_mask = patchify(mask, (patch_size, patch_size, 3), step=patch_size) 
        
                for i in range(patches_mask.shape[0]):
                    for j in range(patches_mask.shape[1]):
                        
                        single_patch_mask = patches_mask[i,j,:,:]
                        single_patch_mask = single_patch_mask[0] # drop the extra unecessary dimension that patchify adds                             
                        mask_dataset.append(single_patch_mask) 

image_dataset = np.array(image_dataset)
mask_dataset =  np.array(mask_dataset)

# Convert HEX to RGB array
Building = '#3C1098'.lstrip('#')
Building = np.array(tuple(int(Building[i:i+2], 16) for i in (0, 2, 4))) # 60, 16, 152

Land = '#8429F6'.lstrip('#')
Land = np.array(tuple(int(Land[i:i+2], 16) for i in (0, 2, 4))) #132, 41, 246

Road = '#6EC1E4'.lstrip('#') 
Road = np.array(tuple(int(Road[i:i+2], 16) for i in (0, 2, 4))) #110, 193, 228

Vegetation =  'FEDD3A'.lstrip('#') 
Vegetation = np.array(tuple(int(Vegetation[i:i+2], 16) for i in (0, 2, 4))) #254, 221, 58

Water = 'E2A929'.lstrip('#') 
Water = np.array(tuple(int(Water[i:i+2], 16) for i in (0, 2, 4))) #226, 169, 41

Unlabeled = '#9B9B9B'.lstrip('#') 
Unlabeled = np.array(tuple(int(Unlabeled[i:i+2], 16) for i in (0, 2, 4))) #155, 155, 155

label = single_patch_mask

# Converts an RGB value to an integer label
def rgb_to_2D_label(label):
    """
    Suply our labale masks as input in RGB format. 
    Replace pixels with specific RGB values ...
    """
    label_seg = np.zeros(label.shape,dtype=np.uint8)
    label_seg [np.all(label == Building,axis=-1)] = 0
    label_seg [np.all(label==Land,axis=-1)] = 1
    label_seg [np.all(label==Road,axis=-1)] = 2
    label_seg [np.all(label==Vegetation,axis=-1)] = 3
    label_seg [np.all(label==Water,axis=-1)] = 4
    label_seg [np.all(label==Unlabeled,axis=-1)] = 5
    
    label_seg = label_seg[:,:,0]  #Just take the first channel, no need for all 3 channels
    
    return label_seg

labels = []
for i in range(mask_dataset.shape[0]):
    label = rgb_to_2D_label(mask_dataset[i])
    labels.append(label)    

labels = np.array(labels)   
labels = np.expand_dims(labels, axis=3)

# Get categorical labels
n_classes = len(np.unique(labels))
from keras.utils import to_categorical
labels_cat = to_categorical(labels, num_classes=n_classes)

# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, labels_cat, test_size = 0.20, random_state = 42)
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

weights = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666, 0.1666] # 1/6 = 0.1666
dice_loss = sm.losses.DiceLoss(class_weights=weights) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss) # loss function is sum of dice loss and focal loss  

metrics = []

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

model = get_model()
model.compile(optimizer='adam', loss=total_loss, metrics=metrics)

Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_001.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_002.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_003.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_004.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_005.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_006.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_007.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation dataset/Tile 1/images/image_part_008.jpg
Now patchifying image: drive/MyDrive/ColabNotebooks/Semantic segmentation datase

In [ ]:
class MyWorker(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
      super().__init__(*args, **kwargs)

    def compute(self, config, budget, **kwargs):
        history = model.fit(X_train, y_train, 
                    batch_size = config['batchsize'], 
                    validation_batch_size = config['valbatchsize'], 
                    verbose=1, 
                    epochs=1, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)
        
        lossarr = history.history['val_loss']
        loss = lossarr[-1]

        return({
                    'loss': float(loss), 
                    'info': {} 
                })
    
    @staticmethod
    def get_configspace():
        config_space = CS.ConfigurationSpace()
        config_space.add_hyperparameter(CS.UniformIntegerHyperparameter('batchsize', lower=1, upper=30))
        config_space.add_hyperparameter(CS.UniformIntegerHyperparameter('valbatchsize', lower=1, upper=30))
        return config_space



In [ ]:
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()
w = MyWorker(sleep_interval = 0, nameserver='127.0.0.1',run_id='example1')
w.run(background=True)
bohb = BOHB(  configspace = w.get_configspace(),
              run_id = 'example1', nameserver='127.0.0.1',
              min_budget=20, max_budget=100
           )
bs_optimized = bohb.run(n_iterations=6)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

522/522 [==============================] - 321s 615ms/step - loss: 0.9478 - val_loss: 0.9184


In [ ]:
id2config = bs_optimized.get_id2config_mapping()
incumbent = bs_optimized.get_incumbent_id()
bs_optimized = id2config[incumbent]['config']['batchsize']
vbs_optimized = id2config[incumbent]['config']['valbatchsize']
print(f"The optimized batch size is {bs_optimized}")
print(f"The optimized validation batch size is {vbs_optimized}")

The optimized batch size is 2
The optimized validation batch size is 29
